In [5]:
import datetime

import fasttext
import keras_core as keras
import numpy as np
import pandas as pd
from keras_core import Sequential
from keras_core.src.layers import Dense
from sklearn.model_selection import train_test_split

## import fastTest model

In [3]:
ft = fasttext.load_model('../../embedding/embedding_data/cc.fr.300.bin')

## Configurations

In [8]:
data_folder_src = '../../data/corpus_for_sexism_french/'
data_path_src = f'{data_folder_src}corpus_with_text_label_final.csv'

date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f"../../log/fit/fasttext_nlp_{date_str}"

model_weight_dst = f'../../model_weights/fasttext_nlp_{date_str}-model.weight'

## loading dataset

In [9]:

ds = pd.read_csv(data_path_src).iloc[:, 1:].dropna()
ds

,full_text,label
1,@MamzelleMNa Une très humble femme! #Ironie,0
2,"BLOGUE - «Tsé, la parité homme-femme...» au se...",0
5,Mise en ligne de mon article sur @PayeTaShnek ...,0
7,Hommage #journeedelafemme Toute sa vie elle a ...,0
8,Lustre bois flotté feuilles exotiques bleues a...,0
...,...,...
9545,"[Exclusif] #BalanceTonPorc, transparence, juge...",1
9547,"Eric Monier, directeur de la rédaction de #LCI...",1
9548,.@annesaurat porte plainte contre son ancien r...,1
9550,"Moïra Sauvage @excisionparlons ""une pensée pou...",1


## text to vector

In [10]:
def text_to_vector(text):
    words = text.split(' ')
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    if not word_vectors:
        return np.zeros(300)
    return np.mean(word_vectors, axis=0)

In [11]:
ds['vector'] = ds['full_text'].apply(text_to_vector)

X = np.vstack(ds['vector'])
y = ds['label'].values

## building and training the model

In [28]:
experiment_name = 'deep_woke_fasttext_classification_mlp'

### callbacks

In [29]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [32]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### training the model

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=300, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=100,
          callbacks=[
              early_stopping,
              tensorboard_callback,
          ],
          validation_data=(X_test, y_test)
          )

Epoch 1/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7170 - loss: 0.6142 - val_accuracy: 0.8812 - val_loss: 0.3760
Epoch 2/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8428 - loss: 0.4463 - val_accuracy: 0.8812 - val_loss: 0.3550
Epoch 3/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8593 - loss: 0.3856 - val_accuracy: 0.8812 - val_loss: 0.3478
Epoch 4/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8502 - loss: 0.3874 - val_accuracy: 0.8812 - val_loss: 0.3363
Epoch 5/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8505 - loss: 0.3772 - val_accuracy: 0.8812 - val_loss: 0.3318
Epoch 6/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8554 - loss: 0.3643 - val_accuracy: 0.8812 - val_loss: 0.3289
Epoch 7/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8597 - loss: 0.3522 - val_accuracy: 0.8812 - val_loss: 0.3263
Epoch 8/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8708 - loss: 0.3225 - val_accuracy: 0.8812 - 

# save model

In [ ]:
model.save_weights(model_weight_dst)

### testing with custom sentence

In [57]:
text_nice = [
    "la bonjour je suis un type sympas", 
    "hello ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"
    
]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.0001}")


print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction =  model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.0001}")

nice output
la bonjour je suis un type sympas : [[False]]
hello ça va les gens : [[False]]
j'aime pas trop ce mec : [[ True]]
cette meuf est cool : [[False]]
les meuf sont trop cool : [[ True]]
les mecs sont des gens stylé : [[ True]]
le mec est stylé : [[False]]
nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse : [[False]]
les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste : [[ True]]
Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions : [[ True]]


not nice output
regarde moi cette connasse : [[ True]]
les femme leur place c'est à la cuisine : [[ True]]
femme au volant, accident : [[ True]]
les femmes on peux pas leur faire confiance : [[ True]]
Pour une transphobe son passing est pas ouf : [[ True]]
Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants : [[ True]]
Dis donc, c’est pas parce que t’es